# Interpolation of a biderectional curve network

In [ ]:
from pygbs import gbs
import numpy as np
from math import radians, cos

In [ ]:
p = 5

# crv_2d = gbs.bscurve2d_approx_from_points("../../../tests/in/e1098.dat",p,gbs.KnotsCalcMode.CHORD_LENGTH,1)
crv_2d = gbs.bscurve2d_approx_from_points("../../../tests/in/e186.dat",p,gbs.KnotsCalcMode.CHORD_LENGTH,1)

nv = 5
r1 = 0.3
r2 = 1.
r  = np.linspace( r1, r2, nv)
v = np.linspace(0., r2-r1, nv)
g  = np.linspace(radians(15), radians(50), nv)
cax= 0.3
l  = np.linspace( 0, 0.3 * cax, nv)
b  = np.linspace( 0, 0.2 * cax, nv)
c  = np.array(list(map(lambda g_ : cax * cos(g_), g)))

def make_3d_foil(params):
    r_, g_, c_, l_, b_ = params
    crv_2d_ = gbs.BSCurve2d(crv_2d)
    gbs.translate(crv_2d_,[-0.5,0.])
    gbs.rotate(crv_2d_, g_)
    gbs.scale(crv_2d_, c_)
    gbs.translate(crv_2d_,[b_,l_])
    gbs.translate(crv_2d_,[0.5,0.])
    return gbs.to_bscurve_3d(crv_2d_, r_)
    
    
foils = list(map(make_3d_foil, zip(r, g, c, l, b)))

u1, u2 = crv_2d.bounds()
nu = 11
u = np.linspace(u1, u2,nu)
q = 3

def make_span_curve(u_):
    pts = [ foil(u_) for foil in foils]
    begin =( v[0], pts[0] )
    end =( v[-1], pts[-1] )
    contraints = [(v_, pt, 0) for v_, pt in zip(v[1:-1], pts[1:-1])]
    contraints.append( (0., [0,0,1], 1) )
    return gbs.interpolate(begin, end, contraints, q)

spans = list(map(make_span_curve, u))


In [ ]:
import pyvista as pv
from pygbs import vistaplot as gbv

# Initialize the PyVista plotter
plotter = pv.Plotter()

# Add the defined curves to the plotter for visualization
gbv.add_curves_to_plotter(foils+spans, plotter)

# Display the plot with the curves
plotter.show()

In [ ]:
Lu = gbs.loftbs(foils, v, q)

plotter = pv.Plotter()
gbv.add_curves_to_plotter(foils, plotter)
gbv.add_surfaces_to_plotter([Lu], plotter, nu = 300, )
plotter.show()

In [ ]:
Lv = gbs.loftbs(spans, u, p)
Lv.invertUV()

plotter = pv.Plotter()
gbv.add_surfaces_to_plotter([Lv], plotter, nu = 300, )
gbv.add_curves_to_plotter(spans, plotter)
plotter.show()

In [ ]:
# This Uniformization works because we are dealing with simple multiplicities
for u_ in Lu.knotsFlatsU():
    if u_ not in Lv.knotsFlatsU():
        Lv.insertKnotU(u_)
for v_ in Lu.knotsFlatsV():
    if v_ not in Lv.knotsFlatsV():
        Lv.insertKnotV(v_)   

for u_ in Lv.knotsFlatsU():
    if u_ not in Lu.knotsFlatsU():
        Lu.insertKnotU(u_)
for v_ in Lv.knotsFlatsV():
    if v_ not in Lu.knotsFlatsV():
        Lu.insertKnotV(v_)  


from pytest import approx

for u1_, u2_ in zip(Lu.knotsFlatsU(), Lv.knotsFlatsU()):
    assert u1_ == approx( u2_ ), f'{u1_=}, {u2_=}'
for v1_, v2_ in zip(Lu.knotsFlatsV(), Lv.knotsFlatsV()):
    assert v1_ == approx( v2_ ), f'{v1_=}, {v2_=}'



In [ ]:
plotter = pv.Plotter()
gbv.add_curves_to_plotter(foils, plotter)
gbv.add_surfaces_to_plotter([Lu], plotter, nu = 300, )
plotter.show()


In [ ]:
plotter = pv.Plotter()
gbv.add_curves_to_plotter(spans, plotter)
gbv.add_surfaces_to_plotter([Lv], plotter, nu = 300, )
plotter.show()

In [ ]:
pts = np.array([ [ foil(u_) for u_ in u] for foil in foils ])
pts = pts.reshape((len(u)*len(v), 3))

flat_u =  gbs.build_simple_mult_flat_knots(u, p)
flat_v =  gbs.build_simple_mult_flat_knots(v, q)

poles = gbs.build_poles(pts, flat_u, flat_v, u, v, p, q)
T = gbs.BSSurface3d(poles, flat_u, flat_v, p, q)

pts = pts.reshape((len(v),len(u), 3))
for j, v_ in enumerate(v):
    for i, u_ in enumerate(u):
        assert T(u_, v_) == approx( pts[j][i] )


# This Uniformization works because we are dealing with simple multiplicities
for u_ in Lu.knotsFlatsU():
    if u_ not in T.knotsFlatsU():
        T.insertKnotU(u_)
for v_ in Lu.knotsFlatsV():
    if v_ not in T.knotsFlatsV():
        T.insertKnotV(v_)   

for u_ in Lv.knotsFlatsU():
    if u_ not in T.knotsFlatsU():
        T.insertKnotU(u_)
for v_ in Lv.knotsFlatsV():
    if v_ not in T.knotsFlatsV():
        T.insertKnotV(v_)  


for u_ in T.knotsFlatsU():
    if u_ not in Lu.knotsFlatsU():
        Lu.insertKnotU(u_)
for v_ in T.knotsFlatsV():
    if v_ not in Lu.knotsFlatsV():
        Lu.insertKnotV(v_)   

for u_ in T.knotsFlatsU():
    if u_ not in Lv.knotsFlatsU():
        Lv.insertKnotU(u_)
for v_ in T.knotsFlatsV():
    if v_ not in Lv.knotsFlatsV():
        Lv.insertKnotV(v_)   

for u1_, u2_, u3_ in zip(Lu.knotsFlatsU(), Lv.knotsFlatsU(), T.knotsFlatsU()):
    assert u1_ == approx( u2_ ), f'{u1_=}, {u2_=}'
    assert u1_ == approx( u3_ ), f'{u1_=}, {u3_=}'
for v1_, v2_, v3_ in zip(Lu.knotsFlatsV(), Lv.knotsFlatsV(), T.knotsFlatsV()):
    assert v1_ == approx( v2_ ), f'{v1_=}, {v2_=}'
    assert v1_ == approx( v3_ ), f'{v1_=}, {v3_=}'

plotter = pv.Plotter()
gbv.add_surfaces_to_plotter([T], plotter, nu = 300, )

plotter.show()

In [ ]:
poles = [ (np.array(pole_u) + np.array(pole_v) - np.array(pole_t)).tolist() for pole_u, pole_v, pole_t in zip(Lu.poles(), Lv.poles(), T.poles())  ]

flat_u = Lu.knotsFlatsU()
flat_v = Lu.knotsFlatsV()

G = gbs.BSSurface3d(poles, flat_u, flat_v, p, q)

plotter = pv.Plotter()
gbv.add_surfaces_to_plotter([G], plotter, nu = 300, per=11,axis=[[0,0,0],[1,0,0]] )
gbv.add_curves_to_plotter(foils, plotter)
gbv.add_curves_to_plotter(spans, plotter)
plotter.show()